1. 문서의 내용을 읽기
2. 문서를 쪼개기
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 쪼갠 문서를 임베딩해서 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 llm에 질문과 함께 전달 

### 문서 내용 읽기

In [1]:
%pip install --upgrade --quiet docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter # 다양한 TextSplitter들중 하나

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 하나의 청크에 가지는 토큰수
    chunk_overlap=200, #  앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정
                       # 가지고 올때 전에 가져온거의 멏줄 정도 겹쳐서 가져옴.(앞뒤 문맥 파악을 위해)
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
len(document_list)

225

### 임베딩

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_openai import OpenAIEmbeddings


# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
# https://platform.openai.com/docs/pricing

크로마는 인메모리인 반면  
파인콘은 aws 클라우드임.  

파인콘은 다른 장소에서 벡터db를 가져오는데 편함.  

RDB에서는 데이터베이스를  
파인콘에서는 인덱스로 부르는데  
5개까지 무료임.  

In [ ]:
%pip install langchain-pinecone

In [5]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

c:\Users\BaekSeungJin\rag-practice\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 답변 생성을 위한 Retrieval

Chroma에 저장한 데이터를 유사도 검색(similarity_search())를 활용해서 가져옴

In [18]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
# 존댓말 쓰면 답변을 좀 더 잘해줌.

In [ ]:
retriever = database.as_retriever(search_kwargs={'k': 3})
retriever.invoke(query)

[Document(id='fbb105bd-1c18-4638-a51a-4698f14d1658', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

### Augmentation을 위한 Prompt 활용

Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')
# https://platform.openai.com/docs/pricing

In [18]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain import hub
# langchain hub는 많은 기능들중 가장 큰 것은
# 효과적인 프롬프트를 준다.
prompt = hub.pull("rlm/rag-prompt")

c:\Users\BaekSeungJin\rag-practice\.venv\Lib\site-packages\langsmith\client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### 답변 생성

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [11]:
# LangChain 권장사항에 따라 강의 코드와 다르게 `.invoke()`를 사용합니다
ai_message = qa_chain.invoke({"query": query})

ai_message

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 84만원에 1,400만원을 초과하는 금액의 15%를 더한 금액입니다. 따라서, 소득세는 84만 원 + (5,000만원 - 1,400만원) × 0.15 = 84만 원 + 510만 원 = 594만 원 정도가 됩니다. 정확한 계산을 위해서는 공제사항 등을 고려해야 합니다.'}

### 결론: 

1. gpt가 소득세 표 이미지를 못 받아왔다.  
    ->  소득세 표 부분만 마크다운으로 바꾸고 다시 임베딩  
        (이때 워드 테이블 사용하면 gpt가 못읽음. 마크다운으로!)    
즉, 데이터 전처리를 잘해주자.  


1. 질문에는 '직장인'이 문서에는 '거주자'가 대상이다.  그래서 retriever했을때 유사도 검색 4번째에 소득세 표 부분이 나온다. (즉 유사도 검색이 잘 안된다.) (k를 3로 하면 안나옴)  
   -> query에 '직장인' 대신 '거주자'로 바꾸기  
즉, 올바르게 질문하자.  

In [12]:
# 하지만 사용자 query는 예상할 수 없음.

### Retrieval을 위한 keyword 사전 활용

사용자 질문을 변경하는 작업을 거치도록 함.  

- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정
- LangChain Expression Language (LCEL)을 활용한 Chain 연계

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [20]:
new_question = dictionary_chain.invoke({"question": query})

In [21]:
new_question

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [16]:
ai_response = tax_chain.invoke({"question": query})

In [17]:
ai_response

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 84만원 plus (1,400만원을 초과하는 금액의 15퍼센트)로 계산됩니다. 즉, 5천만원의 소득에서 1,400만원을 초과한 3,600만원에 15%를 곱하면 540만원이 추가됩니다. 따라서 총 소득세는 84만원 + 540만원으로, 총 624만원입니다.'}